# Explore ethylene oxide data

Ethyelne oxide comes in second in share of the US total RSEI score. It is clear on the map that ethylene oxide facilities are concentrated in Texas and Louisiana. I am going to explore this further here, to find concrete data to cite in my report.

In [1]:
# import packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import mapping
from functools import reduce
import os
import re
import numpy as np

import warnings
warnings.simplefilter(action='ignore')

# supress scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
# read file
ethyl = gpd.read_file('../data/ethylene-facilities.geojson')
ethyl

,YEAR,TRIFD,FACILITY_NAME,STREET_ADDRESS,CITY,COUNTY,ST,ZIP,BIA,TRIBE,LATITUDE,LONGITUDE,INDUSTRY_SECTOR,PARENT_CO_NAME,CHEMICAL,WATER,ON_SITE_RELEASE_TOTAL,AIR,UNIT_OF_MEASURE,geometry
0,2018,70669VSTCHOLDSP,SASOL CHEMICALS (USA) LLC-LAKE CHARLES CHEMICA...,2201 OLD SPANISH TRAIL,WESTLAKE,CALCASIEU PARISH,LA,70669,None,None,30.25880,-93.29370,Chemicals,SASOL (USA) CORP,ETHYLENE OXIDE,0.00000,2237.00000,2237.00000,Pounds,POINT (-93.29370 30.25880)
1,2018,30209CRBRD8195N,BECTON DICKINSON & CO COVINGTON OPERATIONS,8195 INDUSTRIAL BLVD,COVINGTON,NEWTON,GA,30014,None,None,33.60820,-83.83960,Miscellaneous Manufacturing,BECTON DICKINSON & CO,ETHYLENE OXIDE,0.00000,656.30000,656.30000,Pounds,POINT (-83.83960 33.60820)
2,2018,16510MDCLM2205E,COSMED OF PENNSYLVANIA,2205 E 33RD ST,ERIE,ERIE,PA,16510,None,None,42.12398,-80.02186,Miscellaneous Manufacturing,COSMED GROUP INC,ETHYLENE OXIDE,0.00000,293.00000,293.00000,Pounds,POINT (-80.02186 42.12398)
3,2018,74063CHMLN9100W,BAKER PETROLITE LLC,9100 W 21ST ST,SAND SPRINGS,TULSA,OK,74063,None,None,36.12999,-96.09903,Chemicals,BAKER HUGHES A GE CO LLC,ETHYLENE OXIDE,0.00000,9.00000,9.00000,Pounds,POINT (-96.09903 36.12999)
4,2018,49504STRLS520WA,VIANT MEDICAL INC VIANT STERILIZATION SERVICES,520 WATSON ST SW,GRAND RAPIDS,KENT,MI,49504,None,None,42.96175,-85.68245,Miscellaneous Manufacturing,VIANT MEDICAL LLC,ETHYLENE OXIDE,0.00000,378.39000,378.39000,Pounds,POINT (-85.68245 42.96175)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2018,30906KNDLL1816M,KENDALL PATIENT RECOVERY LLC,1430 MARVIN GRIFFIN RD,AUGUSTA,RICHMOND,GA,30906,None,None,33.39440,-81.98460,Miscellaneous Manufacturing,CARDINAL HEALTH INC,ETHYLENE OXIDE,0.00000,259.00000,259.00000,Pounds,POINT (-81.98460 33.39440)
105,2018,29605THXCHPERIM,ETHOX CHEMICALS LLC,1801 PERIMETER RD,GREENVILLE,GREENVILLE,SC,29605,None,None,34.73447,-82.37562,Chemicals,PIEDMONT CHEMICAL INDUSTRIES INC,ETHYLENE OXIDE,0.00000,11.30000,11.30000,Pounds,POINT (-82.37562 34.73447)
106,2018,62525STLYM2200E,TATE & LYLE DECATUR,2200 E ELDORADO ST,DECATUR,MACON,IL,62521,None,None,39.84995,-88.92346,Food,TATE & LYLE INGREDIENTS AMERICAS LLC,ETHYLENE OXIDE,0.00000,239.00000,239.00000,Pounds,POINT (-88.92346 39.84995)
107,2018,43920VNRLL1250S,HERITAGE THERMAL SERVICES,1250 ST GEORGE ST,EAST LIVERPOOL,COLUMBIANA,OH,43920,None,None,40.63162,-80.54632,Hazardous Waste,HERITAGE-WTI LLC,ETHYLENE OXIDE,0.00000,0.00302,0.00302,Pounds,POINT (-80.54632 40.63162)


In [8]:
# count how many facilities per state
ethyl['ST'].value_counts()

TX    25
LA    13
SC     7
WV     6
IL     5
PA     5
NC     4
GA     4
MI     3
PR     3
MO     3
VA     3
WI     3
TN     2
NY     2
MD     2
FL     2
UT     1
NE     1
AZ     1
CO     1
NV     1
KY     1
KS     1
CT     1
IN     1
OH     1
DE     1
CA     1
AR     1
IA     1
NJ     1
MS     1
OK     1
Name: ST, dtype: int64

With 25 facilities, Texas has 23% of all facilities, and Louisiana has 12% with 13 facilities. Now let's see how the release amounts compare.

In [10]:
# find total release amount for all facilities
sumTotal = ethyl['ON_SITE_RELEASE_TOTAL'].sum()
sumTotal

251588.6092

In [16]:
# now aggregate by state to find total amount released by state
stateTotals = ethyl.groupby('ST').agg({'ON_SITE_RELEASE_TOTAL':'sum'
                                               }).reset_index()

stateTotals.sort_values('ON_SITE_RELEASE_TOTAL', ascending=False)

,ST,ON_SITE_RELEASE_TOTAL
29,TX,158975.58398
13,LA,41064.30600
16,MO,8834.00000
33,WV,6941.77000
25,PA,5233.21020
0,AR,4910.62000
5,DE,4886.00000
31,VA,4639.18000
27,SC,3757.84000
3,CO,2467.00000


Texas releases 63.2% of all ethylene oxide, and Lousiana releases 16.32%. In contrast, Missouri comes in third with only 3.5% of all ethylene released.